In [1]:
!pip install --upgrade --quiet pip 
!pip install -I --upgrade --quiet setuptools
!pip install --upgrade --quiet tensorflow edward pymongo
!pip install --quiet lightgbm

In [2]:
% matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

In [55]:
import fetch.fetch as fetch
from edward.models import Normal
from modelling.models import *
from modelling.utils import get_articles
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from pymongo import MongoClient
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
import lightgbm

In [5]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))

In [ ]:
# repro current oracle bug
# collection = MongoClient("209.177.92.45:80")['newsfilter'].news
# l = get_articles(collection, label = True)
# labels = map(lambda x: x['label'], l)
# bodies = map(lambda x: x['content']['body'], l)
# create_model(bodies, labels, [0.5,0.5])

In [126]:
from modelling.clustering import get_unique_items

ge_unique = get_unique_items(df[df._id.str.contains('ge')], .1)
tw_unique = get_unique_items(df[df._id.str.contains('tw')], 0.5)
unique = pd.concat([ge_unique, tw_unique])
v = vectorizer(unique)
tfidf = create_tfidf(v, unique)
word_count = create_word_count(unique).astype('float32')

In [127]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(5, shuffle = True)

def evaluate(model, tfidf, df, method = 'predict_proba'):
    ans = [1 if l == 'accepted' else 0 for l in df.label]
    probs = cross_val_predict(model, tfidf, ans, cv=cv, method=method)
    predictions = cross_val_predict(model, tfidf, ans, cv=cv, method='predict')

    probs = probs if method == 'decision_function' else [p[1] for p in probs] 
                           
    print (average_precision_score(ans, probs), roc_auc_score(ans, probs, average = None))
    print (precision_score(ans, predictions), recall_score(ans, predictions), fbeta_score(ans, predictions, beta = 1.5))
    # precision, recall, _ = precision_recall_curve(ans, guess)
    fpr, tpr, _ = roc_curve(ans, probs)
    plt.step(fpr, tpr)
    # plt.step(precision, recall)
    return probs, ans, predictions

In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
grid = GridSearchCV(lightgbm.LGBMClassifier(n_estimators = 100, reg_lambda = 0.6, reg_alpha = 0.2), parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, unique.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [ ]:
model = lightgbm.LGBMClassifier(n_estimators = 100, reg_lambda = 0.6, reg_alpha = 0.2)
_ = evaluate(model, word_count, unique)

In [ ]:
model.fit(word_count, df.label)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
g, ans, preds = evaluate(model, word_count, unique)

In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
model = svm.LinearSVC(dual = False, class_weight = 'balanced', C = 4., tol = 10e-6, max_iter = 4000)
grid = GridSearchCV(model, parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, unique.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [192]:
from sklearn import svm

model = svm.LinearSVC(dual = False,  tol = 10e-6, max_iter = 4000, class_weight = 'balanced')
g,a,p = evaluate(model, word_count, unique, 'decision_function')

0.818509120245 0.880303234501
0.835714285714 0.668571428571 0.712412177986


In [ ]:
model = svm.OneClassSVM()
g,a,p = evaluate(model, tfidf, unique, 'decision_function')

In [130]:
model = MultinomialNB(alpha = .001, class_prior = [.5,.5])
g, ans, preds = evaluate(model, word_count, unique)

0.789992610346 0.856590296496
0.417553191489 0.897142857143 0.662877557649


In [ ]:
model.fit(tfidf, df.label)

# model.feature_log_prob_
sorted(zip(model.feature_log_prob_[0], v.get_feature_names()))[-50:]